In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler

In [3]:
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [9]:
class DataLoader:
    def __init__(self):
        AS_dataset = pd.read_csv('./Arbitrary_Single_band_Coupler_Phase_Shift.csv', encoding='utf-8')
        self.X = AS_dataset.loc[:,'freq':'L4'].to_numpy()
        self.y = AS_dataset.loc[:,'S11r':'S41i'].to_numpy()
#         self.mmX = MinMaxScaler()
#         self.X[:,1:] = self.mmX.fit_transform(self.X[:,1:])
#         self.X[:,0] = self.X[:,0] / 10
#         self.X, _, self.y, _ = train_test_split(self.X, self.y, test_size=0.75, random_state=0)
        self.X_train, self.X_vali, self.y_train, self.y_vali = train_test_split(self.X, self.y, test_size=0.1, random_state=0)
        self.num_train = self.X_train.shape[0]
    def get_batch(self, batch_size=0, mode='train'):
        if mode == 'train':
            index = np.random.randint(0, self.num_train, batch_size)
            return self.X_train[index], self.y_train[index]
        if mode == 'validate':
            return self.X_vali, self.y_vali

In [10]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)
    
#     @tf.function
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [11]:
num_epochs = 200
batch_size = 1024
learning_rate = 0.001

In [12]:
model = MLP()
data_loader = DataLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
X_v, y_v = data_loader.get_batch(mode='validate')
def train():
    num_batch = data_loader.num_train // batch_size
    for epoch_index in range(num_epochs):
        for batch in range(num_batch):
            X, y = data_loader.get_batch(batch_size)
            with tf.GradientTape() as tape:
                y_pred = model(X)
                tr_mse = tf.reduce_mean(tf.square(y_pred - y))
            grads = tape.gradient(tr_mse, model.variables)
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        if epoch_index % 10 == 0 or epoch_index == num_epochs - 1:
            tr_rmse = tf.sqrt(tr_mse)
            tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
            tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.cast(tf.reduce_mean(y), dtype=tf.float32)))
            print("epoch:{}".format(epoch_index))
            print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
            y_v_p = model(X_v)
            va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
            va_rmse = tf.sqrt(va_mse)
            va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
            va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.cast(tf.reduce_mean(y_v), dtype=tf.float32)))
            print("vali mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [13]:
train()

epoch:0
train mse:0.0952557697892189 rmse:0.30863532423973083 mae:0.25238871574401855 r2:0.21212852001190186
vali mse:0.09371825307607651 rmse:0.3061343729496002 mae:0.24976204335689545 r2:0.22309726476669312
epoch:10
train mse:0.07130761444568634 rmse:0.2670348584651947 mae:0.21519824862480164 r2:0.40854591131210327
vali mse:0.07203582674264908 rmse:0.26839491724967957 mae:0.21656323969364166 r2:0.4028396010398865
epoch:20
train mse:0.06365450471639633 rmse:0.2522984445095062 mae:0.20191535353660583 r2:0.47458916902542114
vali mse:0.06593257933855057 rmse:0.2567733824253082 mae:0.2049683779478073 r2:0.4534341096878052
epoch:30
train mse:0.061959683895111084 rmse:0.24891701340675354 mae:0.19569209218025208 r2:0.4868156909942627
vali mse:0.05900618061423302 rmse:0.24291187524795532 mae:0.19354911148548126 r2:0.5108523368835449
epoch:40
train mse:0.05336909368634224 rmse:0.23101751506328583 mae:0.1824861466884613 r2:0.5586906671524048
vali mse:0.05311031639575958 rmse:0.23045675456523895

In [33]:
tf.saved_model.save(model, './models')

INFO:tensorflow:Assets written to: ./models\assets


In [14]:
model = tf.saved_model.load('./models')

In [42]:
def obj_func(s_para):
    E = tf.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    l1 = E11 - E21 - E31 + E41
    l2 = tf.square(E21 / (E31 + E21) - 2 / 3)
    l3 = tf.square(tf.math.atan2(s_para[:,5], s_para[:,4]) - tf.math.atan2(s_para[:,3], s_para[:,2]) - np.pi / 4)
    l4 = tf.square(tf.reduce_sum(E, axis=1) - 1)
    loss = l1 + l2 + l3 + l4
    return loss

In [43]:
num_nodes = 5000
num_node_epochs = 200

In [44]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)

mmin = np.min(data_loader.X[: , 1: ], axis=0)
mmax = np.max(data_loader.X[: , 1: ], axis=0)

# structure.append(tf.Variable(np.random.uniform(0, 1, (num_nodes, 10)), dtype=tf.float32))
structure = tf.Variable(np.random.uniform(mmin, mmax, (num_nodes, 12)), dtype=tf.float32)

freq1 = tf.ones([num_nodes, 1]) * 2.4
freq2 = tf.ones([num_nodes, 1]) * 2.5
freq3 = tf.ones([num_nodes, 1]) * 2.6

minLoss = 0
minIndex = 0
minS = 0

In [45]:
bestLoss = 10
bestStructure = 0

In [47]:
def check(structure):
    inva_place1 = tf.where(tf.logical_or(structure[:,:8] < 1, structure[:,:8] > 4))
    structure = tf.tensor_scatter_nd_update(structure, [inva_place1], [np.random.uniform(mmin[inva_place1[:,1]], mmax[inva_place1[:,1]], (inva_place1.shape[0]))])
    
    inva_place2 = tf.where(tf.logical_or(structure[:,8:] < 4, structure[:,8:] > 100)) + [0, 8]
    structure = tf.tensor_scatter_nd_update(structure, [inva_place2], [np.random.uniform(mmin[inva_place2[:,1]], mmax[inva_place2[:,1]], (inva_place2.shape[0]))])
    
    return tf.Variable(structure)
    
    # structure[j] = tf.Variable(tf.tensor_scatter_nd_update(structure[j], [nega_place], [np.random.uniform(0, 1, (nega_place.shape[0]))]))

In [48]:
for i in range(num_node_epochs):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(structure)
        y_pred1 = model(tf.concat([freq1, structure], axis=1))
        y_pred2 = model(tf.concat([freq2, structure], axis=1))
        y_pred3 = model(tf.concat([freq3, structure], axis=1))
        loss = obj_func(y_pred1) + obj_func(y_pred2) + obj_func(y_pred3)
    minLoss = tf.reduce_min(loss).numpy()
    minIndex = tf.argmin(loss).numpy()
    minS = structure[minIndex].numpy()
    grads = tape.gradient(loss, structure)
    opt.apply_gradients(grads_and_vars=zip([grads], [structure]))
    structure = check(structure)
    if minLoss < bestLoss:
        bestLoss = minLoss
        bestStructure = minS
        # bestStructure = data_loader.mmX.inverse_transform([minS[0]])[0]
        print(minIndex)
        print(i, bestLoss)
        print(bestStructure)
        print()

2230
0 -1.9859238
[ 4.5723305  2.991286   2.854786   4.19759    4.4924245  1.4546661
  1.377169   2.9322083 98.22275   36.799522  10.596915  78.14789  ]

2230
1 -2.0379958
[ 2.0177522  2.981289   2.8447857  2.9113207  3.4707549  1.444666
  1.3671688  2.9222083 98.212746  36.80952   10.586916  78.137886 ]

2230
2 -2.0502336
[ 2.0251935  2.98873    2.8373444  2.9038794  3.478196   1.4372249
  1.3597274  2.9296496 98.220184  36.816963  10.594357  78.13045  ]

2230
3 -2.0606916
[ 2.0315814  2.9951177  2.8309562  2.8974915  3.4845839  1.4308369
  1.3533393  2.9360375 98.22657   36.823353  10.600744  78.12406  ]

2230
4 -2.070194
[ 2.0373926  3.0009286  2.825145   2.8916802  3.4903948  1.4250258
  1.347528   2.9418485 98.23238   36.829163  10.606555  78.11825  ]

2230
5 -2.0792317
[ 2.0428474  3.0063832  2.8196902  2.8862255  3.4958496  1.419571
  1.3420731  2.9473033 98.23784   36.834618  10.612009  78.11279  ]

2230
6 -2.0878568
[ 2.0480685  3.011604   2.814469   2.8810043  3.5010705  1.41

2230
54 -2.4427204
[ 2.3272417  3.1006615  2.5352917  2.6018302  3.7802377  1.1351839
  1.0576726  3.2316945 98.52225   37.119007  10.89639   78.05809  ]

2230
55 -2.4506793
[ 2.334581   3.0933225  2.5279522  2.594491   3.787577   1.1278449
  1.0503331  3.2390337 98.52959   37.126347  10.9037285 78.06543  ]

2230
56 -2.4586778
[ 2.3419824  3.0859213  2.5205505  2.5870895  3.7949784  1.1204436
  1.0429314  3.2464352 98.53699   37.133747  10.91113   78.07283  ]

2230
57 -2.4667268
[ 2.3494458  3.078458   2.5130868  2.579626   3.8024418  1.1129804
  1.0354677  3.2538986 98.54445   37.14121   10.918593  78.08029  ]

2230
58 -2.4748008
[ 2.3569705  3.0709333  2.5055618  2.572101   3.8099668  1.1054558
  1.0279427  3.2614236 98.55197   37.148735  10.926118  78.087814 ]

2230
59 -2.482864
[ 2.3645563  3.0633476  2.4979758  2.564515   3.8175528  1.0978701
  1.0203565  3.2690096 98.559555  37.156322  10.933703  78.0954   ]

2230
60 -2.490395
[ 2.3722029  3.0709937  2.4903293  2.5568686  3.82519